In [21]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_spd_matrix  #to generate covariance matrices
from numpy.linalg import det
from scipy.stats import special_ortho_group
import collections
from numpy.random import dirichlet
from scipy.stats import invwishart, matrix_normal
from numpy.linalg import inv,det
from numpy import sqrt

seed = 123
np.random.seed(seed)

In [22]:
#uploading data
meno_data = 10
X_data = np.load('./simulation/x.npy')[:,:meno_data]
z_data = np.load('./simulation/z.npy')[:meno_data]

K = np.unique(z_data).size
M = X_data.shape[0]
T = X_data.shape[1]

## INFERENCE AND GIBBS SAMPLER

$\pi_k$ is the $k^{th}$ row of the transition matrix and it is a vector containing the probabilities to reach another discrete state starting from $z = k$.\
Thus, it is a vector of the type: $\pi_k = (p_{k,0}, p_{k,1},\dots,p_{k,K-2},p_{k,K-1})$. 

The problem we are facing is conceptually similar to the *mixture of gaussians model* apart from the fact that there are $K$ vectors like $\pi_k$, while in the *mixture of gaussians model* there is only one.
So, we can write $P(z_t = q | z_{t-1} = k) = p_{k,q}$ or $P(z_t | z_{t-1} = k) = cat(\pi_k)$ for all $k$s. 

We choose as prior for every $\pi_k$ a Dirichlet distribution i.e. the conjugate prior of the categorical:
$\boldsymbol{\pi}_k | \alpha_k \sim \operatorname{Dir}\left(\boldsymbol{\alpha}_k\right)$ with $\boldsymbol{\alpha_{k}} = \boldsymbol{1} \in \mathbb{R}^{K}$ for an uniformative prior.\
In this way the posterior is again a Dirichlet: 
$P(\boldsymbol{\pi}_k | z_t, z_{t-1} = k) = \operatorname{Dir}\left(\boldsymbol{\alpha}_k + \boldsymbol{n}_k \right)$
where $\boldsymbol{n}_k = (n_{k,0}, \dots, n_{k,K-2}, n_{k,K-1})$ is the vector containing the number of times is observed a transition $k \rightarrow q$ for every $q$.

It looks like the whole procedure is an hybrid between *soft clustering* and *multivariate bayesian regression*: given all the experimental data $(\boldsymbol{X},\boldsymbol{Y})$ we want to divide them in $K$ clusters obtaining
$(\boldsymbol{X^{(k)}},\boldsymbol{Y^{(k)}})\; \forall k$. Each cluster is defined by two matrices $A_k$ and $Q_k$ and we want to perform a linear regression on them to find the matrices. Eventually we also want the probability that the next point will be sampled from a given cluster i.e. the transition matrix and the distribution of the variable $z_t$.

Let's go go on with the $K$ linear regression (one for each cluster). It is worth to notice is that the linear regreassions are separated: $X^{(k)}$ does not interact with $X^{(k-1)}$. What I mean is that at this point the math of the problem says that we are going to perform linear regressions on $k$ systems of the form: $X_{s}^{(k)} = A_K X_{s - 1}^{(k)} + b_k$, where the time $s$ is redifined on the subset $X^{(k)}$ of $X$.
We want to perform the regressions in homogeneous coordinates, so the dynamics become $Y_{s}^{(k)} = A_K X_{s - 1}^{(k)}$, but now $Y_{s}^{(k)} \in \mathbb{R}^{M}$, $X_{s - 1}^{(k)} \in \mathbb{R}^{M + 1}$ and $A_{k} \in \mathbb{R}^{M x (M + 1)}$ and its last column is the vector $b_k$.

Let's write something more on the bayesian linear regression. I am dropping the index $k$ but it must be understood that everything is done for the $k^{th}$ bayesian linear regression.
We choose as priors: $P(A|Q) = \mathcal{M} \mathcal{N}_{M,M+1}(M_0,Q,\Lambda_{0}^{-1})$ where $M_0 \in \mathbb{R}^{M,M + 1}$ and $\Lambda_{0}^{-1} \in \mathbb{R}^{M + 1,M + 1}$ and also $P(Q) = \mathcal{W}^{-1}\left(\mathbf{V}_0, \boldsymbol{\nu}_0\right)$
where $\mathbf{V}_0$ has the same dimensions as $Q$ ($Q \in \mathbb{R}^{MxM}$) and it is positive definite, $\boldsymbol{\nu}_0$ is a number.
Using the priors above, the posteriors are:

$$
P(\boldsymbol{Q} |\mathbf{Y}, \mathbf{X}) \sim \mathcal{W}^{-1}\left(\mathbf{V}_T, \nu_T\right)\\
P(\mathbf{A} | \mathbf{Y}, \mathbf{X}, \mathbf{Q}) \sim \mathcal{M} \mathcal{N}_{M, M+1}\left(\mathbf{M}_T,\mathbf{Q}, \mathbf{\Lambda}_T^{-1}\right)
$$

where:
\begin{aligned}
& \mathbf{V}_T=\mathbf{V}_0+\left(\mathbf{Y}-\mathbf{M_T X}\right)\left(\mathbf{Y}-\mathbf{M_T X}\right)^{\top} + \left(\mathbf{M}_T -\mathbf{M}_0\right) \mathbf{\Lambda}_0\left(\mathbf{M}_T -\mathbf{M}_0\right)^{\top} \\
& \boldsymbol{\nu}_T =\boldsymbol{\nu}_0 + T \\
& \mathbf{M}_T^{\top} = (\mathbf{X} \mathbf{X}^{\top}+\mathbf{\Lambda}_0)^{-1} (\mathbf{X} \mathbf{Y}^{\top} +\mathbf{\Lambda}_0 \mathbf{M}_0^{\top})\\
& \mathbf{\Lambda}_T=\mathbf{X} \mathbf{X}^{\top} + \mathbf{\Lambda}_0
\end{aligned}

(In this way it should be ok, it is different from wikipedia 'bayesian multivariate linear regression' because they start with a different equation for the linear regression).
Where $T$ is actually $T_k$ and it is the number of columns f the matrix $Y^{(k)}$.\
What make sense to speed up the convergence of the Gibbs sampler is to compute $\Lambda_{T}$ using itself at the previous interation instead of $\Lambda_{0}$ and the same for the other parameters.\
Furthermore is convenient to firstly compute $\nu_T$ and $\Lambda_T$ and then plague them in the other to parameters.

Eventually we have to sample the the whole vector $z_t$ from its posterior:

$$
\operatorname{P}\left(z_t=k | z_{t-1},ALL\right)=\frac{r_{t k}}{\sum_k r_{t k}} \; \text{where} \;
r_{t k}=p_{z_{t-1},k}\left|Q_k\right|^{-1 / 2} \exp \left\{-\frac{1}{2}\left(\mathbf{y}_t -A_k \mathbf{x}_{t}\right)^T Q_k^{-1}\left(\mathbf{y}_t - A_k \mathbf{x}_{t} \right)^T \right\}\\
\text{or equivalentely} \; z_t \sim \operatorname{cat}\left(p_t^{\prime}\right) \; \text{where} \; p_t^{\prime} = \operatorname{P}\left(z_t | z_{t-1},ALL\right) \; \forall t
$$

In some sense it is a probability that takes into account the distance between the cluster labeled $k$ and the point $x_t$.
The whole procedure must be carried on till convergence.







In [23]:
#preparing data in homogeneous coordinates
X = X_data[:, 0:T-1] #this X_{t-1} so times run between [0,T-1]

#attaching ones
X = np.vstack((X,np.ones(T-1))) #dimensions (M+1) x (T-1)

Y = X_data[:, 1:T] #this is X_t so times run between [1,T], dimensions M x (T-1)

n_iterations = 15 #number of iterations in the Gibbs sampler


In [24]:
#let's start mfs.The code below must be reorganised and the loops in k grouped

z = np.zeros((n_iterations, T - 1), dtype = np.int8)

#uniform random vector z_t of length T
z[0,:] = np.random.choice(K, size = T - 1, p = None) #for each couple (X_t,Y_t) there must be a value of z_t

#N_k is a matrix KxK containing as rows the vectors n_k
N_k = np.zeros((K,K))

#filling the matrix N_k
for i in range(T - 2):
    N_k[z[0,i]][z[0,i+1]] += 1 

#now I label the experimental point based on the extracted z_t, will do it as a vocabolary of matrices.
X_k = {}
Y_k = {} #it is X_k translated of one time-step (without homogeneous coordinates)
for k in np.arange(K):
    
    k_indices = np.where(z[0,:] == k)
    X_k[str(k)] = X[:,k_indices[0]] #k_indices[0] otherwise python adds a void extra_dimension
    Y_k[str(k)] = Y[:,k_indices[0]]

#now I need to initialise nu_k, L_k, V_k, A_k, M_k
#we need dictionaries of dictionaries: first index being k and the second the number of the iteration s.
Q_k = collections.defaultdict(dict)
L_k = collections.defaultdict(dict)
V_k = collections.defaultdict(dict)
nu_k = collections.defaultdict(dict)
A_k = collections.defaultdict(dict)
M_k = collections.defaultdict(dict)


for k in np.arange(K): #I do not really understand the meaning of this initialization
    nu_k[str(k)]['0'] = K #greater or equal then the dimension of the scale matrix
    L_k[str(k)]['0'] = make_spd_matrix(M+1, random_state = seed) #np.ones((M+1,M+1)) questo non funziona perchè è singolare
    V_k[str(k)]['0'] = make_spd_matrix(M, random_state = seed) #this must be positive semidefinite
    M_k[str(k)]['0'] = np.zeros((M,M+1))

#now as in 'details' randomly sample the transition matrix, Q_k, A_k from the priors 
Q_k = collections.defaultdict(dict)

PI_k = collections.defaultdict(dict)

#parameter for the dirichlet
alp = np.ones(K) 

for k in np.arange(K): #Each dictionary of PI_k correspond to the same k and so to the same row of the transition matrix.
    PI_k[str(k)]['0'] = dirichlet(alpha = alp, size = 1).flatten()
    
    Q_k[str(k)]['0'] = invwishart.rvs(df = nu_k[str(k)]['0'], scale =  V_k[str(k)]['0'])
    
    A_k[str(k)]['0'] = matrix_normal.rvs(mean = M_k[str(k)]['0'], colcov = inv(L_k[str(k)]['0']), rowcov = Q_k[str(k)]['0'] )
    
#Now we have a first approximation of the transition matrix, of the matrices A_k, Q_k and the other parameters.
#Let's start the iterations

for it in range(1, n_iterations):
    for k in np.arange(K):
         PI_k[str(k)][str(it)] = dirichlet(alpha = alp + N_k[k,:], size = 1).flatten()
            
         #now I have to update L_k, nu_k, A_k, V_k, Q_k
         L_k[str(k)][str(it)] = X_k[str(k)] @ X_k[str(k)].T +  L_k[str(k)][str(it - 1)]
         nu_k[str(k)][str(it)] = nu_k[str(k)][str(it-1)] + Y_k[str(k)].shape[1]
        
         #I am using L_k[k][it] to update M_k, I think this will speed up the convergence but it is only a guess
         M_k[str(k)][str(it)] = inv(L_k[str(k)][str(it)]) @\
        (X_k[str(k)] @ Y_k[str(k)].T + L_k[str(k)][str(it)] @ M_k[str(k)][str(it-1)].T)
        
         M_k[str(k)][str(it)] = M_k[str(k)][str(it)].T

         V_k[str(k)][str(it)] = V_k[str(k)][str(it - 1)] + (Y_k[str(k)] - M_k[str(k)][str(it)] @ X_k[str(k)]) @\
         (Y_k[str(k)] - M_k[str(k)][str(it)] @ X_k[str(k)]).T + (M_k[str(k)][str(it)] - M_k[str(k)][str(it-1)])@\
         L_k[str(k)][str(it)] @ (M_k[str(k)][str(it)] - M_k[str(k)][str(it-1)]).T
        
         Q_k[str(k)][str(it)] = invwishart.rvs(df = nu_k[str(k)][str(it)], scale =  V_k[str(k)][str(it)])
         A_k[str(k)][str(it)] = matrix_normal.rvs(mean = M_k[str(k)][str(it)], colcov = inv(L_k[str(k)][str(it)]), \
                                              rowcov = Q_k[str(k)][str(it)] )
    #now sampling z from the posterior
    for t in np.arange(1,T-2):
        
        #Pi_k[str(k)][...][...] is the probability to go from the state z_{t-1} to the state z_t = k
        
        p_prime = np.array([ PI_k[str(k)][str(it)][z[it,t-1]]/sqrt(det(Q_k[str(k)][str(it)])) *\
                            np.exp((-0.5 * (Y[:,t] - A_k[str(k)][str(it)] @ X[:,t]).T \
                                    @ inv(Q_k[str(k)][str(it)]) @\
                                    (Y[:,t] - A_k[str(k)][str(it)] @ X[:,t]))) for k in np.arange(K)])
        
        p_prime = p_prime/np.sum(p_prime)
        
        z[it,t] = np.random.choice(K, size = 1, p = p_prime)
    
    #recomputing stuff
    #not finished

In [26]:
print(z[:,:])
print(p_prime)

[[2 1 2 2 0 2 2 1 2]
 [0 1 2 2 0 2 2 1 0]
 [0 1 2 0 0 2 2 1 0]
 [0 0 2 0 0 2 2 1 0]
 [0 1 2 2 1 2 1 1 0]
 [0 1 2 1 0 1 1 1 0]
 [0 1 2 0 0 2 1 1 0]
 [0 1 2 2 0 2 1 2 0]
 [0 0 2 2 0 2 2 1 0]
 [0 2 0 2 0 2 1 1 0]
 [0 2 2 0 0 2 1 1 0]
 [0 0 2 2 0 2 1 1 0]
 [0 0 2 0 0 2 1 1 0]
 [0 2 2 1 2 1 2 1 0]
 [0 2 2 2 0 2 1 1 0]]
[1.63332006e-26 6.79074172e-01 3.20925828e-01]
